## Commands and notes

In [ ]:
# oomstopper --no-tail combinasons ; killbill combinasons ; cd ~/twinews-logs ; jupython -o nohup-combinasons-$HOSTNAME.out --venv st-venv ~/Workspace/Python/Datasets/Twinews/twinews/models/combinasons.ipynb

In [ ]:
# TODO convertir "doc2vec" en "doc2vec-hash" --> en cherchant le meilleur ndcg
# Pour le parametre qui determine les modèles utilisé pour le merge

## Init

In [1]:
import os ; os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [2]:
isNotebook = '__file__' not in locals()

In [3]:
TEST = isNotebook # isNotebook, True, False

In [4]:
from systemtools.hayj import *
from systemtools.location import *
from systemtools.basics import *
from systemtools.file import *
from systemtools.printer import *
from nlptools.preprocessing import *
from nlptools.basics import *
from twinews.utils import *
from twinews.models.ranking import *
from machinelearning.iterator import *

In [5]:
from twinews.models.genericutils import *
from twinews.models.ranking import *

In [6]:
logger = Logger(tmpDir('logs') + "/combinasons.log") if isNotebook else Logger("combinasons-" + getHostname() + ".log")
tt = TicToc(logger=logger)
tt.tic()

--> tictoc starts...


-1

## Config

In [7]:
config = \
{
    'splitVersion': 2,
    'maxUsers': 2 if TEST else None, # Sub-sampling
}

--> tic: 32.03s | message: Eval data loaded
--> toc total duration: 32.3s | message: Got Twinews evaluation data


In [43]:
def getBestModelKey(model, splitVersion, maxUsers=None, metric='ndcg'):
    twinewsRankings = getTwinewsRankings(verbose=False)
    twinewsScores = getTwinewsScores(verbose=False)
    keys = twinewsRankings.keys()
    rows = []
    for key in keys:
        meta = twinewsRankings.getMeta(key)
        if meta['splitVersion'] == splitVersion \
        and meta['maxUsers'] == None \
        and meta['model'] == model:
            scoreRow = twinewsScores.findOne({'id': meta['id'], 'metric': metric})
            assert 'score' not in meta
            meta['score'] = scoreRow['score']
            rows.append(meta)
    rows = [(e['id'], e['score']) for e in rows]
    rows = sortBy(rows, index=1, desc=True)
    best = rows[0][0]
    return best

In [47]:
def getRankings(model, userIds=None):
    twinewsRankings = getTwinewsRankings(verbose=False)
    rankings = twinewsRankings[model]
    assert rankings is not None
    if userIds is not None:
        rankings = dictSelect(rankings, userIds)
    return rankings

In [50]:
rankings1 = getRankings(getBestModelKey("doc2vec", 2), userIds=userIds)
rankings2 = getRankings(getBestModelKey("dbert-ft", 2), userIds=userIds)

In [56]:
userId = list(userIds)[0]
rankings1_u = rankings1[userId][0]
rankings2_u = rankings2[userId][0]

In [57]:
bp(rankings1_u, 4)

[
  ( http://ow.ly/QYCH50g4Ut7, 0.5698583872633665 ),
  (
    https://www.tennessean.com/story/news/2018/01/04/mayor-barrys-transit-plan-balloons-9-b-nearly-double-construction-price-tag-when-totaling-other-cost/1004229001/,
    0.6631916299649248
  ),
  (
    http://buffalonews.com/2018/01/08/state-pulls-plug-on-scajaquada-plan/,
    0.666428096491031
  ),
  ( http://ow.ly/isIK30hFWG4, 0.6708006289376947 ),
  ( http://bit.ly/2lk30rI, 0.679991741383384 ),
  ( https://buff.ly/2D849NU, 0.6825276236905211 ),
  ( http://bit.ly/2AOmdYi, 0.684166999809006 ),
  (
    http://thehill.com/policy/transportation/366621-five-obstacles-to-trumps-infrastructure-ambitions,
    0.6868035645824813
  ),
  ( http://ow.ly/Ppis30hELRV, 0.6872319722368091 ),
  (
    http://www.startribune.com/opinion/commentaries/468684353.html,
    0.688717367754799
  ),
  ...,
  ( https://usat.ly/2CwKJ68, 0.906982549653865 ),
  ( https://usat.ly/2Dljtoi, 0.9085274194595565 ),
  (
    https://food.ndtv.com/food-drinks/heres

In [63]:
bp(rankings2_u, 4)

[
  ( http://ow.ly/QYCH50g4Ut7, 0.14096046404639498 ),
  ( https://buff.ly/2D849NU, 0.21665274231308684 ),
  ( http://ow.ly/q8DE50g2NXU, 0.2166629748151577 ),
  ( http://ow.ly/Uo4Z50fV0Uc, 0.24395355309716177 ),
  (
    https://www.greeleytribune.com/news/local/rising-groundwater-in-the-past-decade-inundates-more-basements-each-year-costing-homeowners-from-gilcrest-to-sterling-thousands-of-dollars-each/,
    0.24964305539802886
  ),
  ( http://ow.ly/W3mY50g03YZ, 0.2817210466917336 ),
  ( http://nyti.ms/2D9RoCJ, 0.2866143637929417 ),
  (
    https://www.nytimes.com/2018/01/03/technology/2018-technology-chaos.html?smprod=nytcore-ipad&smid=nytcore-ipad-share,
    0.30288912483019587
  ),
  ( https://buff.ly/2CIywqJ, 0.3111657847489136 ),
  ( https://nyti.ms/2pRRWrq, 0.318873731687528 ),
  ...,
  ( https://usat.ly/2lCbbQg, 0.6952691043485183 ),
  (
    http://www.wildcat.arizona.edu/article/2018/01/sean-miller-basketball-arizona-colorado,
    0.6989488747350779
  ),
  (
    http://www.dail

In [67]:
rankings = mergeRankings\
(
    [rankings1_u, rankings2_u],
    rankAsScore=[True, True],
    weights=[0.1, 0.9],
    alphas=[0.5, 0.5],
    betas=None,
)

{
  'http://a.msn.com/01/en-ie/BBHzIAS?ocid=st': [ 0.0871871871871872, 0.7351351351351352 ],
  'http://abcn.ws/2Dwr16b': [ 0.05035035035035035, 0.5 ],
  'http://abcn.ws/2mAeZ66': [ 0.0897897897897898, 0.40270270270270275 ],
  'http://abcn.ws/2zWkQGp': [ 0.08628628628628629, 0.7765765765765765 ],
  'http://abcnews.go.com/Politics/trumps-hole-comments-travel-ban-30/story?id=52306560': [ 0.05195195195195195, 0.6954954954954955 ],
  'http://bangordailynews.com/2018/01/08/opinion/editorials/theres-no-evidence-of-voter-fraud-in-maine-so-lets-stop-wasting-time-on-it/': [ 0.024324324324324322, 0.23153153153153153 ],
  'http://basketball.dailyherald.com/sports/20180111/progress-gives-glenbard-east-edge-over-west-chicago': [ 0.06656656656656657, 0.7918918918918919 ],
  'http://bbc.in/2D6Cdra': [ 0.07277277277277278, 0.8081081081081081 ],
  'http://bbc.in/2DCRxw6': [ 0.014914914914914913, 0.5855855855855856 ],
  'http://bbc.in/2qcwugI': [ 0.09279279279279279, 0.6864864864864865 ],
  'http://bit.l

In [68]:
bp(rankings, 4, logger)

[
  ( http://ow.ly/QYCH50g4Ut7, 0.0 ),
  ( https://buff.ly/2D849NU, 0.001401401401401381 ),
  ( http://ow.ly/q8DE50g2NXU, 0.0054054054054053615 ),
  ( http://nyti.ms/2D9RoCJ, 0.006506506506506483 ),
  (
    https://www.greeleytribune.com/news/local/rising-groundwater-in-the-past-decade-inundates-more-basements-each-year-costing-homeowners-from-gilcrest-to-sterling-thousands-of-dollars-each/,
    0.006506506506506526
  ),
  ( http://ow.ly/Uo4Z50fV0Uc, 0.016216216216216252 ),
  ( http://ow.ly/Vlbj30hG1q8, 0.018318318318318316 ),
  ( http://bit.ly/2CFdo4B, 0.01961961961961961 ),
  ( https://fb.me/2liBWd5L2, 0.0244244244244244 ),
  ( https://nyti.ms/2E5Rbgs, 0.026026026026025974 ),
  ...,
  (
    http://www.whas11.com/news/local/stolen-lmpd-cruiser-recovered/504647136,
    0.9676676676676678
  ),
  ( http://bit.ly/2Cybh6W, 0.96996996996997 ),
  (
    http://www.dailymail.co.uk/news/article-5252995/Michael-Douglas-denies-sexual-harassment-claims.html,
    0.9705705705705707
  ),
  (
    htt

In [69]:
userIdsCache = None

In [81]:
def generateRankings(config, logger=None, verbose=True):
    # We init user ids cache:
    global userIdsCache
    if userIdsCache is None:
        userIdsCache = dict()
    # We get models:
    assert "models" in config
    assert isinstance(config["models"], set)
    assert len(config["models"]) >= 2
    models = config["models"]
    models = sorted(list(models))
    # We set default params:
    if not dictContains(config, 'rankAsScore'):
        config['rankAsScore'] = [True] * len(models)
        logWarning("You didn't set rankAsScore", logger=logger, verbose=verbose)
    if not dictContains(config, 'weights'):
        config['weights'] = [1.0 / len(models)] * len(models)
        logWarning("You didn't set weights", logger=logger, verbose=verbose)
    if not dictContains(config, 'alphas'):
        config['alphas'] = [0.5] * len(models)
        logWarning("You didn't set alphas", logger=logger, verbose=verbose)
    if not dictContains(config, 'betas'):
        config['betas'] = [NormalizedLawBeta.LOG] * len(models)
        logWarning("You didn't set betas", logger=logger, verbose=verbose)
    # We get user ids:
    userIds = None
    if config["maxUsers"] is not None:
        if config["maxUsers"] in userIdsCache:
            userIds = userIdsCache[config["maxUsers"]]
        else:
            evalData = getEvalData(config['splitVersion'], maxExtraNews=0, maxUsers=config['maxUsers'],
                                   logger=logger, verbose=verbose)
            userIds = set(evalData['candidates'].keys())
            log("Users: " + b(userIds), logger)
            userIdsCache[config["maxUsers"]] = userIds
    # We get best models:
    models = sorted([getBestModelKey(model, config['splitVersion']) for model in models])
    log("Setting models as: " + b(models), logger)
    config['models'] = models
    # We get rankings:
    rankings = []
    for model in models:
        rankings.append(getRankings(model, userIds=userIds))
    bp(rankings)

In [85]:
currentConfig = mergeDicts\
(
    config,
    {
        'models': {'doc2vec', 'dbert-ft'},
        'rankAsScore': [True, True],
        'weights': [0.5, 0.5],
        'alphas': [0.5, 0.5],
        'betas': [NormalizedLawBeta.LOG, NormalizedLawBeta.LOG],
    }
)

In [86]:
generateRankings(currentConfig, logger=logger)

Setting models as: [ dbert-ft-d1b5f, doc2vec-19ca3 ]
